In [1]:
!pip install earthengine-api geemap matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 20.9 MB/s eta 0:00:00


In [2]:
import ee
ee.Authenticate()

In [3]:
import ee
import geemap
import os
import matplotlib.pyplot as plt

# --- 1. Inisialisasi Google Earth Engine ---
# Pastikan Anda sudah menjalankan ee.Authenticate() sebelumnya
try:
    ee.Initialize(project='ee-prabowows')
    print("Google Earth Engine berhasil diinisialisasi.")
except Exception as e:
    print(f"Error inisialisasi GEE: {e}")
    print("Pastikan Anda sudah menjalankan 'ee.Authenticate()' dan login.")
    exit()

Google Earth Engine berhasil diinisialisasi.


In [4]:
# --- 2. Tentukan Area of Interest (AOI) ---
# Koordinat bbox untuk wilayah Kudus, Jawa Tengah, Indonesia
# Anda bisa mendapatkan koordinat ini dari Google Maps atau sumber lainnya
# Format: [longitude_min, latitude_min, longitude_max, latitude_max]
#kudus_bbox = ee.Geometry.BBox(110.8, -6.88, 111.0, -6.65) # Perkiraan bbox untuk Kudus

# Anda bisa juga membuat titik dan buffer di sekitarnya jika diinginkan
kudus_point = ee.Geometry.Point(128.143056, 1.048611) # Pusat Kota Kudus
aoi = kudus_point.buffer(50000) # Buffer 50 km

#aoi = kudus_bbox # Kita gunakan bbox yang lebih luas untuk contoh ini
print(f"AOI yang digunakan: {aoi.getInfo()}")


# --- 3. Tentukan Parameter Pencarian Citra ---
start_date = '2024-01-01'
end_date = '2024-12-31' # Mencari citra sepanjang tahun 2024
cloud_filter = 10 # Maksimum persentase tutupan awan (0-100)

print(f"\nMencari citra Sentinel-2 dari {start_date} sampai {end_date} dengan tutupan awan < {cloud_filter}%")

# --- 4. Mengambil Koleksi Citra Sentinel-2 ---
# Koleksi: 'COPERNICUS/S2_SR_HARMONIZED' adalah Sentinel-2 Surface Reflectance (Level-2A)
# yang sudah dikoreksi atmosferik dan diharmonisasi.
collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_filter))

# Menampilkan jumlah citra yang ditemukan
count = collection.size().getInfo()
print(f"Jumlah citra yang ditemukan: {count}")

if count == 0:
    print("Tidak ada citra yang sesuai dengan kriteria filter. Coba perluas tanggal atau tingkatkan cloud_filter.")
else:
    # --- 5. Membuat Komposit Citra (Median Komposit) ---
    # Mengambil median dari semua piksel dalam koleksi untuk mengurangi efek awan dan variasi lain.
    # Pilih band-band yang relevan untuk visualisasi (True Color: B4=Red, B3=Green, B2=Blue)
    # Atau False Color (B8=NIR, B4=Red, B3=Green) untuk melihat vegetasi lebih jelas

    # Untuk visualisasi True Color (B4, B3, B2)
    composite_image_rgb = collection.median().select(['B4', 'B3', 'B2'])

    # Untuk visualisasi False Color (B8, B4, B3) - lebih baik untuk melihat vegetasi
    composite_image_false_color = collection.median().select(['B8', 'B4', 'B3'])

    # Klip citra agar hanya fokus pada AOI
    clipped_rgb = composite_image_rgb.clip(aoi)
    clipped_false_color = composite_image_false_color.clip(aoi)

    # --- 6. Visualisasi (menggunakan geemap) ---
    print("\nVisualisasi citra (akan muncul di jendela browser atau notebook)...")

    # Hitung centroid AOI untuk memusatkan peta
    # FIX: Tambahkan maxError=1 ke panggilan centroid()
    center_lat = aoi.centroid(maxError=1).coordinates().get(1).getInfo()
    center_lon = aoi.centroid(maxError=1).coordinates().get(0).getInfo()

    map_viz = geemap.Map(center=[center_lat, center_lon], zoom=11)

    # Parameter visualisasi untuk True Color
    vis_params_rgb = {
        'min': 0,
        'max': 3000, # Sesuaikan nilai max jika citra terlalu gelap/terang
        'bands': ['B4', 'B3', 'B2']
    }

    # Parameter visualisasi untuk False Color
    vis_params_false_color = {
        'min': 0,
        'max': 5000, # Sesuaikan nilai max
        'bands': ['B8', 'B4', 'B3']
    }

    map_viz.addLayer(clipped_rgb, vis_params_rgb, 'Sentinel-2 True Color (Kudus)')
    map_viz.addLayer(clipped_false_color, vis_params_false_color, 'Sentinel-2 False Color (Kudus)', False) # Nonaktifkan secara default
    map_viz.addLayer(aoi, {'color': 'red', 'fillColor': '00000000'}, 'Area of Interest') # Tambahkan fill color transparan

    map_viz.add_basemap('Esri.WorldImagery') # Tambahkan basemap citra satelit
    # Untuk colorbar, biasanya Anda hanya perlu satu set parameter untuk band yang relevan
    map_viz.add_colorbar(vis_params_rgb, label="Nilai Piksel (True Color)", orientation="vertical", layer_name="Sentinel-2 True Color (Kudus)")

    map_viz.set_plot_options(add_marker_cluster=True, add_data_gui=True, add_layer_manager=True)
    map_viz # Menampilkan peta di Jupyter Notebook/Google Colab

    # --- 7. Ekspor Citra (Opsional: untuk analisis lebih lanjut di luar GEE) ---
    # Perlu diingat bahwa ekspor ke Google Drive bisa memakan waktu tergantung ukuran citra.
    # Anda harus memiliki akun Google Drive yang terhubung dengan akun Google Anda.
    export_to_drive = True # Set False jika tidak ingin mengekspor

    if export_to_drive:
        print("\nMulai proses ekspor citra ke Google Drive...")

        # Export True Color
        task_rgb = ee.batch.Export.image.toDrive(
            image=clipped_rgb,
            description='Sentinel2_Kudus_TrueColor_2024',
            folder='GEE_Exports', # Folder di Google Drive Anda
            fileNamePrefix='kudus_sentinel2_rgb_2024',
            scale=10, # Resolusi output (10 meter untuk Sentinel-2)
            region=aoi, # FIX: Lewati objek ee.Geometry secara langsung
            maxPixels=1e10 # Tingkatkan jika area terlalu besar
        )
        task_rgb.start()
        print(f"Ekspor True Color ke Google Drive dimulai. Status: {task_rgb.status()['state']}. Cek Task Manager di Earth Engine Code Editor.")

        # Export False Color
        task_false_color = ee.batch.Export.image.toDrive(
            image=clipped_false_color,
            description='Sentinel2_Kudus_FalseColor_2024',
            folder='GEE_Exports',
            fileNamePrefix='kudus_sentinel2_falsecolor_2024',
            scale=10,
            region=aoi, # FIX: Lewati objek ee.Geometry secara langsung
            maxPixels=1e10
        )
        task_false_color.start()
        print(f"Ekspor False Color ke Google Drive dimulai. Status: {task_false_color.status()['state']}. Cek Task Manager di Earth Engine Code Editor.")

        # Anda bisa memeriksa status tugas secara manual di Earth Engine Code Editor (tab Tasks).
        # Atau secara programatis (contoh sederhana, bisa lebih kompleks untuk multi-task):
        # import time
        # print("\nMenunggu ekspor selesai (ini bisa memakan waktu)...")
        # while task_rgb.active() or task_false_color.active():
        #     if task_rgb.active():
        #         print(f"Status ekspor RGB: {task_rgb.status()['state']}")
        #     if task_false_color.active():
        #         print(f"Status ekspor False Color: {task_false_color.status()['state']}")
        #     time.sleep(30) # Tunggu 30 detik sebelum memeriksa lagi
        # print(f"Ekspor RGB selesai dengan status: {task_rgb.status()['state']}")
        # print(f"Ekspor False Color selesai dengan status: {task_false_color.status()['state']}")


    print("\nProses selesai.")

AOI yang digunakan: {'type': 'Polygon', 'coordinates': [[[128.14305600000003, 1.4985542975740747], [128.01734003590778, 1.4806436846223894], [127.90163613795853, 1.428338352574301], [127.80515744531023, 1.345803928915109], [127.73558371144806, 1.2396127146811524], [127.69845020093068, 1.1182195516231463], [127.69670837134959, 0.9912881509329452], [127.73049262690834, 0.8689217393444477], [127.79711096164814, 0.7608592117317615], [127.89125980711226, 0.6757005190050142], [128.00544601252946, 0.6202226637883989], [128.1305827086199, 0.5988405908037805], [128.25671197726794, 0.6132558935195538], [128.37379694339856, 0.6623213963992877], [128.4725201643249, 0.742132452024311], [128.54502466720933, 0.8463376438888033], [128.58553873933704, 0.9666440643760068], [128.5908350612044, 1.09347691293505], [128.56048800288033, 1.2167410362236901], [128.49690871307402, 1.3266240630047115], [128.4051548665182, 1.4143774824847368], [128.29252950280602, 1.4730135947462615], [128.16800015794277, 1.49786

In [5]:
map_viz.set_plot_options(add_marker_cluster=True, add_data_gui=True, add_layer_manager=True)
map_viz # Menampilkan peta di Jupyter Notebook/Google Colab


Map(center=[1.0486622923514413, 128.14305742264443], controls=(WidgetControl(options=['position', 'transparent…

# **Land Cover Changes Analysis**

In [6]:
# --- 2. Tentukan Area of Interest (AOI) ---
kudus_point = ee.Geometry.Point(128.143882, 1.048953) # Pusat Kota Kudus
aoi = kudus_point.buffer(2000) # Buffer 2 km
print(f"AOI yang digunakan: {aoi.getInfo()}")


# --- 3. Tentukan Parameter Pencarian Citra dan Analisis ---
# Daftar tahun yang ingin Anda analisis
years_to_analyze = [2020, 2021, 2022, 2023, 2024]
cloud_filter = 10 # Maksimum persentase tutupan awan (0-100)
number_of_clusters = 5 # Jumlah kategori tutupan lahan yang ingin diidentifikasi (misalnya, 5: air, vegetasi padat, vegetasi jarang, area terbangun, lahan kosong)

print(f"\nMulai analisis tutupan lahan untuk tahun-tahun: {years_to_analyze}")
print(f"Dengan filter awan < {cloud_filter}% dan {number_of_clusters} kategori tutupan lahan.")

# Inisialisasi peta geemap
# Hitung centroid AOI untuk memusatkan peta
center_lat = aoi.centroid(maxError=1).coordinates().get(1).getInfo()
center_lon = aoi.centroid(maxError=1).coordinates().get(0).getInfo()
map_viz = geemap.Map(center=[center_lat, center_lon], zoom=11)
map_viz.add_basemap('Esri.WorldImagery') # Tambahkan basemap citra satelit
map_viz.addLayer(aoi, {'color': 'red', 'fillColor': '00000000'}, 'Area of Interest')


# --- Loop Melalui Setiap Tahun untuk Analisis Tutupan Lahan ---
for year in years_to_analyze:
    print(f"\nMemproses tahun: {year}")
    start_date_year = f'{year}-01-01'
    end_date_year = f'{year}-12-31'

    # 4. Mengambil Koleksi Citra Sentinel-2 untuk tahun tertentu
    collection_year = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(aoi) \
        .filterDate(start_date_year, end_date_year) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_filter))

    count_year = collection_year.size().getInfo()
    print(f"Jumlah citra Sentinel-2 ditemukan untuk tahun {year}: {count_year}")

    if count_year == 0:
        print(f"Tidak ada citra yang sesuai untuk tahun {year}. Melanjutkan ke tahun berikutnya.")
        continue # Lewati tahun ini jika tidak ada data

    # 5. Membuat Komposit Citra (Median Komposit)
    # Pilih band yang relevan untuk klasifikasi
    # Band umum untuk klasifikasi: B2 (Biru), B3 (Hijau), B4 (Merah), B8 (NIR), B11 (SWIR1), B12 (SWIR2)
    bands_for_classification = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
    composite_image = collection_year.median().select(bands_for_classification)

    # Klip citra agar hanya fokus pada AOI
    clipped_image = composite_image.clip(aoi)

    # 6. Melakukan Klasifikasi Tanpa Supervisi (Unsupervised Classification) menggunakan K-Means
    # Melatih clusterer
    # Mengambil sampel piksel dari citra untuk melatih clusterer
    training = clipped_image.sample(
        region=aoi,
        scale=10, # Resolusi sampel
        numPixels=500, # Jumlah piksel yang diambil sampelnya
        seed=0, # Untuk hasil yang dapat direproduksi
        geometries=True # Mempertahankan geometri piksel sampel
    )

    # Inisialisasi clusterer K-Means
    clusterer = ee.Clusterer.wekaKMeans(number_of_clusters).train(training, bands_for_classification)

    # Klasifikasikan citra menggunakan clusterer yang telah dilatih
    classified_image = clipped_image.cluster(clusterer)

    # 7. Visualisasi Hasil Klasifikasi
    # Membuat palet warna acak untuk visualisasi cluster
    # Ini akan memberikan warna berbeda untuk setiap cluster, tetapi warnanya acak
    # dan Anda perlu menginterpretasikan sendiri kategori tutupan lahannya.
    palette = ['FF0000', '00FF00', '0000FF', 'FFFF00', 'FF00FF', '00FFFF', 'FFA500', '800080', '008000', 'FFC0CB']
    # Pastikan palet cukup panjang untuk jumlah cluster Anda
    vis_params_classified = {
        'min': 0,
        'max': number_of_clusters - 1,
        'palette': palette[:number_of_clusters] # Ambil sebanyak cluster yang dibutuhkan
    }

    map_viz.addLayer(classified_image, vis_params_classified, f'Land Cover {year} ({number_of_clusters} Clusters)')
    print(f"Layer tutupan lahan untuk tahun {year} ditambahkan ke peta.")

# Menampilkan peta akhir dengan semua layer
map_viz.set_plot_options(add_marker_cluster=True, add_data_gui=True, add_layer_manager=True)
map_viz # Menampilkan peta di Jupyter Notebook/Google Colab

# --- Opsi Ekspor Hasil Klasifikasi (Opsional) ---
# Anda bisa menambahkan bagian ini di dalam loop 'for year'
# jika Anda ingin mengekspor setiap peta tutupan lahan tahunan.
export_classified_to_drive = True # Set True jika ingin mengekspor hasil klasifikasi

if export_classified_to_drive:
    print("\nMulai proses ekspor citra klasifikasi ke Google Drive...")
    for year in years_to_analyze:
        # Ulangi proses pengambilan dan klasifikasi citra untuk memastikan objek 'classified_image' tersedia
        # (atau simpan hasil klasifikasi dalam dictionary jika Anda tidak ingin mengulang komputasi)
        start_date_year = f'{year}-01-01'
        end_date_year = f'{year}-12-31'
        collection_year = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
            .filterBounds(aoi) \
            .filterDate(start_date_year, end_date_year) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_filter))

        if collection_year.size().getInfo() == 0:
            continue

        composite_image = collection_year.median().select(bands_for_classification)
        clipped_image = composite_image.clip(aoi)
        training = clipped_image.sample(
            region=aoi, scale=10, numPixels=500, seed=0, geometries=True
        )
        clusterer = ee.Clusterer.wekaKMeans(number_of_clusters).train(training, bands_for_classification)
        classified_image_to_export = clipped_image.cluster(clusterer)

        task_classified = ee.batch.Export.image.toDrive(
            image=classified_image_to_export.byte(), # Konversi ke byte jika cluster ID adalah integer kecil
            description=f'LandCover_Kudus_{year}_{number_of_clusters}Clusters',
            folder='GEE_LandCover_Exports',
            fileNamePrefix=f'kudus_landcover_{year}_{number_of_clusters}clusters',
            scale=10,
            region=aoi,
            maxPixels=1e10
        )
        task_classified.start()
        print(f"Ekspor tutupan lahan untuk tahun {year} dimulai. Cek Task Manager di Earth Engine Code Editor.")

print("\nAnalisis tutupan lahan selesai. Periksa peta di atas untuk visualisasi.")

AOI yang digunakan: {'type': 'Polygon', 'coordinates': [[[128.143882, 1.066950731902967], [128.1388541223157, 1.0662344181132017], [128.13422647021645, 1.06414249627529], [128.1303674092712, 1.0608414859255333], [128.12758412173324, 1.0565941508389751], [128.12609815453425, 1.0517385819756937], [128.12602778503515, 1.0466612844747218], [128.1273786079253, 1.0417664112737521], [128.13004309198445, 1.0374435925310332], [128.13380914133273, 1.0340369216197052], [128.13837697915625, 1.0318175662299547], [128.1433830097026, 1.0309621844443686], [128.14842875921715, 1.0315388635548257], [128.15311259251837, 1.033501700650264], [128.15706168116162, 1.0366944562600544], [128.15996167913093, 1.0408629902911732], [128.16158174432027, 1.0456754905762675], [128.16179291424686, 1.050748884145174], [128.1605783730947, 1.0556793291852813], [128.15803479233765, 1.0600743607380565], [128.154364637581, 1.0635841313781957], [128.1498600533943, 1.0659292599779808], [128.1448796085429, 1.066923071511403], 

In [7]:
# Menampilkan peta akhir dengan semua layer
map_viz.set_plot_options(add_marker_cluster=True, add_data_gui=True, add_layer_manager=True)
map_viz # Menampilkan peta di Jupyter Notebook/Google Colab

Map(center=[1.0489550517736121, 128.14388205691156], controls=(WidgetControl(options=['position', 'transparent…

# **Land Cover Analysis Dengan True Color**

In [8]:
# --- 2. Tentukan Area of Interest (AOI) ---
kudus_point = ee.Geometry.Point(128.143882, 1.048953) # Pusat Kota Kudus
aoi = kudus_point.buffer(2000) # Buffer 2 km
print(f"AOI yang digunakan: {aoi.getInfo()}")


# --- 3. Tentukan Parameter Pencarian Citra dan Analisis ---
years_to_analyze = [2020, 2021, 2022, 2023, 2024]
cloud_filter = 40 # Maksimum persentase tutupan awan (0-100)
number_of_clusters = 2

print(f"\nMulai analisis tutupan lahan untuk tahun-tahun: {years_to_analyze}")
print(f"Dengan filter awan < {cloud_filter}% dan {number_of_clusters} kategori tutupan lahan.")

# --- Inisialisasi Peta geemap untuk Visualisasi ---
center_lat = aoi.centroid(maxError=1).coordinates().get(1).getInfo()
center_lon = aoi.centroid(maxError=1).coordinates().get(0).getInfo()

map_viz = geemap.Map(center=[center_lat, center_lon], zoom=11)
map_viz.add_basemap('Esri.WorldImagery') # Tambahkan basemap citra satelit
map_viz.addLayer(aoi, {'color': 'red', 'fillColor': '00000000'}, 'Area of Interest')

# Inisialisasi palet warna untuk klasifikasi
palette_classified = [
    'FF0000', '00FF00', '0000FF', 'FFFF00', 'FF00FF', '00FFFF',
    'FFA500', '800080', '008000', 'FFC0CB'
]

# Parameter visualisasi untuk True Color (mentah)
vis_params_rgb_raw = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2'] # True Color Bands
}

# --- Loop Melalui Setiap Tahun untuk Analisis dan Visualisasi ---
for year in years_to_analyze:
    print(f"\nMemproses tahun: {year}")
    start_date_year = f'{year}-01-01'
    end_date_year = f'{year}-12-31'

    collection_year = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(aoi) \
        .filterDate(start_date_year, end_date_year) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_filter))

    count_year = collection_year.size().getInfo()
    print(f"Jumlah citra Sentinel-2 ditemukan untuk tahun {year}: {count_year}")

    if count_year == 0:
        print(f"Tidak ada citra yang sesuai untuk tahun {year}. Melanjutkan ke tahun berikutnya.")
        continue

    # Membuat Komposit Citra (Median Komposit)
    # Citra True Color (mentah)
    composite_image_rgb = collection_year.median().select(['B4', 'B3', 'B2'])
    clipped_rgb = composite_image_rgb.clip(aoi)

    # Citra untuk Klasifikasi
    bands_for_classification = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
    composite_image_classified = collection_year.median().select(bands_for_classification)
    clipped_image_classified = composite_image_classified.clip(aoi)

    # Melakukan Klasifikasi Tanpa Supervisi (Unsupervised Classification)
    training = clipped_image_classified.sample(
        region=aoi, scale=10, numPixels=500, seed=0, geometries=True
    )
    clusterer = ee.Clusterer.wekaKMeans(number_of_clusters).train(training, bands_for_classification)
    classified_image = clipped_image_classified.cluster(clusterer)

    # --- Visualisasi Hasil ---
    # Tambahkan layer citra True Color (mentah) untuk tahun ini
    # Set 'False' agar layer ini tidak aktif secara default
    map_viz.addLayer(clipped_rgb, vis_params_rgb_raw, f'Sentinel-2 True Color {year}', False)
    print(f"Layer True Color untuk tahun {year} ditambahkan ke peta.")

    # Tambahkan layer hasil klasifikasi untuk tahun ini
    vis_params_classified = {
        'min': 0,
        'max': number_of_clusters - 1,
        'palette': palette_classified[:number_of_clusters]
    }
    map_viz.addLayer(classified_image, vis_params_classified, f'Land Cover {year} ({number_of_clusters} Clusters)', False) # Set 'False'
    print(f"Layer tutupan lahan untuk tahun {year} ditambahkan ke peta.")


# --- Tampilkan Peta Akhir dengan Semua Layer ---
map_viz.set_plot_options(add_marker_cluster=True, add_data_gui=True, add_layer_manager=True)
map_viz # Menampilkan peta di Jupyter Notebook/Google Colab

print("\nProses selesai. Peta interaktif akan muncul di atas.")
print("Gunakan tombol 'Layers' di peta untuk mengaktifkan/menonaktifkan citra dan hasil klasifikasi per tahun.")


AOI yang digunakan: {'type': 'Polygon', 'coordinates': [[[128.143882, 1.066950731902967], [128.1388541223157, 1.0662344181132017], [128.13422647021645, 1.06414249627529], [128.1303674092712, 1.0608414859255333], [128.12758412173324, 1.0565941508389751], [128.12609815453425, 1.0517385819756937], [128.12602778503515, 1.0466612844747218], [128.1273786079253, 1.0417664112737521], [128.13004309198445, 1.0374435925310332], [128.13380914133273, 1.0340369216197052], [128.13837697915625, 1.0318175662299547], [128.1433830097026, 1.0309621844443686], [128.14842875921715, 1.0315388635548257], [128.15311259251837, 1.033501700650264], [128.15706168116162, 1.0366944562600544], [128.15996167913093, 1.0408629902911732], [128.16158174432027, 1.0456754905762675], [128.16179291424686, 1.050748884145174], [128.1605783730947, 1.0556793291852813], [128.15803479233765, 1.0600743607380565], [128.154364637581, 1.0635841313781957], [128.1498600533943, 1.0659292599779808], [128.1448796085429, 1.066923071511403], 

In [9]:
# Menampilkan peta akhir dengan semua layer
map_viz.set_plot_options(add_marker_cluster=True, add_data_gui=True, add_layer_manager=True)
map_viz # Menampilkan peta di Jupyter Notebook/Google Colab

Map(center=[1.0489550517736121, 128.14388205691156], controls=(WidgetControl(options=['position', 'transparent…

In [11]:
import pandas as pd

# --- 2. Tentukan Area of Interest (AOI) ---
kudus_point = ee.Geometry.Point(128.143882, 1.048953) # Pusat Kota Kudus
aoi = kudus_point.buffer(2000) # Buffer 2 km
print(f"AOI yang digunakan: {aoi.getInfo()}")


# --- 3. Tentukan Parameter Pencarian Citra dan Analisis ---
years_to_analyze = [2020, 2021, 2022, 2023, 2024] # Ganti dengan tahun yang Anda inginkan
cloud_filter = 10 # Maksimum persentase tutupan awan (0-100)
number_of_clusters = 2 # Jumlah kategori K-Means

print(f"\nMulai analisis 'tanah coklat' untuk tahun-tahun: {years_to_analyze}")
print(f"Dengan filter awan < {cloud_filter}% dan {number_of_clusters} kategori K-Means.")


# --- BAGIAN PENTING: MAPPING CLUSTER KE 'TANAH COKLAT' UNTUK SETIAP TAHUN ---
brown_land_cluster_ids_per_year = {
    2020: [0, 1], # >>> GANTI INI dengan ID cluster 'tanah coklat' untuk tahun 2020 <<<
    2021: [0, 1],    # >>> GANTI INI dengan ID cluster 'tanah coklat' untuk tahun 2021 <<<
    2022: [0, 1], # >>> GANTI INI dengan ID cluster 'tanah coklat' untuk tahun 2022 <<<
    2023: [0, 1],    # >>> GANTI INI dengan ID cluster 'tanah coklat' untuk tahun 2023 <<<
    2024: [0, 1], # >>> GANTI INI dengan ID cluster 'tanah coklat' untuk tahun 2024 <<<
}


# --- Inisialisasi Peta geemap untuk Visualisasi ---
center_lat = aoi.centroid(maxError=1).coordinates().get(1).getInfo()
center_lon = aoi.centroid(maxError=1).coordinates().get(0).getInfo()

map_viz = geemap.Map(center=[center_lat, center_lon], zoom=11)
map_viz.add_basemap('Esri.WorldImagery')
map_viz.addLayer(aoi, {'color': 'red', 'fillColor': '00000000'}, 'Area of Interest')

vis_params_rgb_raw = {
    'min': 0,
    'max': 3000,
    'bands': ['B4', 'B3', 'B2']
}
vis_params_brown_land = {
    'min': 0,
    'max': 1,
    'palette': ['green', 'brown']
}
palette_classified_kmeans = [
    'FF0000', '00FF00', '0000FF', 'FFFF00', 'FF00FF', '00FFFF',
    'FFA500', '800080', '008000', 'FFC0CB'
]


brown_land_areas_sqkm = {}
all_cluster_areas_sqkm = {}

# --- Loop Melalui Setiap Tahun untuk Analisis dan Visualisasi ---
for year in years_to_analyze:
    print(f"\nMemproses tahun: {year}")
    start_date_year = f'{year}-01-01'
    end_date_year = f'{year}-12-31'

    collection_year = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED') \
        .filterBounds(aoi) \
        .filterDate(start_date_year, end_date_year) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_filter))

    count_year = collection_year.size().getInfo()
    print(f"Jumlah citra Sentinel-2 ditemukan untuk tahun {year}: {count_year}")

    if count_year == 0:
        print(f"Tidak ada citra yang sesuai untuk tahun {year}. Melanjutkan ke tahun berikutnya.")
        brown_land_areas_sqkm[year] = 0
        all_cluster_areas_sqkm[year] = {f'Cluster {i}': 0 for i in range(number_of_clusters)}
        continue

    composite_image_rgb = collection_year.median().select(['B4', 'B3', 'B2'])
    clipped_rgb = composite_image_rgb.clip(aoi)

    bands_for_classification = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
    composite_image_classified = collection_year.median().select(bands_for_classification)
    clipped_image_classified = composite_image_classified.clip(aoi)

    training = clipped_image_classified.sample(
        region=aoi, scale=10, numPixels=500, seed=0, geometries=True
    )
    clusterer = ee.Clusterer.wekaKMeans(number_of_clusters).train(training, bands_for_classification)
    classified_image = clipped_image_classified.cluster(clusterer)

    # --- HITUNG LUAS TIAP CLUSTER DI SINI (SOLUSI UNTUK ERROR) ---
    print(f"  Menghitung luas per cluster untuk tahun {year}...")
    pixel_area_sqm = 10 * 10 # Luas satu piksel Sentinel-2 dalam meter persegi (100 m^2)

    # Pastikan band classified_image bernama 'cluster' untuk referensi yang jelas
    classified_image = classified_image.rename('cluster')

    # Buat citra baru: band pertama adalah 'pixel_count' (untuk dijumlahkan),
    # band kedua adalah 'cluster' (untuk dikelompokkan/groupField)
    image_for_reduction = ee.Image.constant(1).rename('pixel_count').addBands(classified_image)

    # Menggunakan Reducer.sum().group()
    cluster_area_raw = image_for_reduction.reduceRegion(
        reducer=ee.Reducer.sum().group(groupField=1, groupName='cluster_id'), # groupField=1 karena 'cluster' sekarang di indeks 1
        geometry=aoi,
        scale=10,
        maxPixels=1e10,
        tileScale=4 # Penting untuk komputasi besar
    ).getInfo()

    current_year_cluster_areas = {}
    total_area_calculated_for_clusters = 0

    if 'groups' in cluster_area_raw:
        for group in cluster_area_raw['groups']:
            cluster_id = group['cluster_id']
            pixel_count = group['sum'] # Reducer.sum() akan memberikan total count dari band 'pixel_count'
            area_sqkm = (pixel_count * pixel_area_sqm) / 1e6
            current_year_cluster_areas[f'Cluster {cluster_id}'] = area_sqkm
            total_area_calculated_for_clusters += area_sqkm

    for i in range(number_of_clusters): # Pastikan semua cluster_id ada, bahkan yang luasnya 0
        if f'Cluster {i}' not in current_year_cluster_areas:
            current_year_cluster_areas[f'Cluster {i}'] = 0

    all_cluster_areas_sqkm[year] = current_year_cluster_areas
    print(f"  Luas per Cluster (km²) untuk tahun {year}: {current_year_cluster_areas}")
    print(f"  Total Area Dihitung (sum of clusters): {total_area_calculated_for_clusters:.2f} km²")


    # --- Re-klasifikasi untuk Mengidentifikasi 'Tanah Coklat' ---
    brown_ids = brown_land_cluster_ids_per_year.get(year, [])

    if not brown_ids:
        print(f"  Tidak ada ID cluster 'tanah coklat' yang ditentukan untuk tahun {year}. Anggap 0 km².")
        binary_brown_land = ee.Image(0).rename('brown_land').clip(aoi)
    else:
        all_cluster_ids = list(range(number_of_clusters))
        remap_values = [1 if cluster_id in brown_ids else 0 for cluster_id in all_cluster_ids]
        binary_brown_land = classified_image.remap(
            all_cluster_ids,
            remap_values
        ).rename('brown_land').byte()

    # --- Hitung Luas Area 'Tanah Coklat' ---
    brown_pixels = binary_brown_land.eq(1)
    brown_pixel_count = brown_pixels.reduceRegion(
        reducer=ee.Reducer.sum(), # Cukup sum karena kita hanya menghitung piksel bernilai 1
        geometry=aoi,
        scale=10,
        maxPixels=1e10,
        tileScale=4
    ).getInfo()

    area_sqkm_brown_land = (brown_pixel_count.get('brown_land', 0) * pixel_area_sqm) / 1_000_000
    brown_land_areas_sqkm[year] = area_sqkm_brown_land

    print(f"  Luas 'Tanah Coklat' untuk tahun {year}: {area_sqkm_brown_land:.2f} km²")


    # --- Visualisasi Hasil ---
    map_viz.addLayer(clipped_rgb, vis_params_rgb_raw, f'S2 RGB {year}', False)
    map_viz.addLayer(classified_image, {'min': 0, 'max': number_of_clusters - 1, 'palette': palette_classified_kmeans[:number_of_clusters]}, f'K-Means Clusters {year}', False)
    map_viz.addLayer(binary_brown_land, vis_params_brown_land, f'Tanah Coklat {year}', False)


# --- Tampilkan Peta Akhir dengan Semua Layer ---
map_viz.set_plot_options(add_marker_cluster=True, add_data_gui=True, add_layer_manager=True)
map_viz

# --- Tampilkan Hasil Luas Area Tanah Coklat Tahunan dalam Tabel ---
print("\n--- Ringkasan Luas Area 'Tanah Coklat' dari Tahun ke Tahun ---")
df_brown_land_areas = pd.DataFrame.from_dict(brown_land_areas_sqkm, orient='index', columns=['Luas (km²)'])
print(df_brown_land_areas)

# --- Tampilkan Hasil Luas Area Per Cluster dari Tahun ke Tahun ---
print("\n--- Ringkasan Luas Area Tiap Cluster (K-Means Asli) dari Tahun ke Tahun (km²) ---")
df_all_cluster_areas = pd.DataFrame.from_dict(all_cluster_areas_sqkm, orient='index')
df_all_cluster_areas = df_all_cluster_areas.reindex(columns=[f'Cluster {i}' for i in range(number_of_clusters)], fill_value=0)
print(df_all_cluster_areas)


print("\nProses selesai.")

AOI yang digunakan: {'type': 'Polygon', 'coordinates': [[[128.143882, 1.066950731902967], [128.1388541223157, 1.0662344181132017], [128.13422647021645, 1.06414249627529], [128.1303674092712, 1.0608414859255333], [128.12758412173324, 1.0565941508389751], [128.12609815453425, 1.0517385819756937], [128.12602778503515, 1.0466612844747218], [128.1273786079253, 1.0417664112737521], [128.13004309198445, 1.0374435925310332], [128.13380914133273, 1.0340369216197052], [128.13837697915625, 1.0318175662299547], [128.1433830097026, 1.0309621844443686], [128.14842875921715, 1.0315388635548257], [128.15311259251837, 1.033501700650264], [128.15706168116162, 1.0366944562600544], [128.15996167913093, 1.0408629902911732], [128.16158174432027, 1.0456754905762675], [128.16179291424686, 1.050748884145174], [128.1605783730947, 1.0556793291852813], [128.15803479233765, 1.0600743607380565], [128.154364637581, 1.0635841313781957], [128.1498600533943, 1.0659292599779808], [128.1448796085429, 1.066923071511403], 